In [1]:
import os

# Define the target directory
target_directory = (
    r"C:\Users\pablosal\Desktop\gbbai-azure-aoai-faq"  # change your directory here
)

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-azure-aoai-faq


In [1]:
# Import the libraries
from dotenv import load_dotenv
from src.aoai.azure_openai import AzureOpenAIManager
from typing import Dict, Optional
from requests import Response
import requests

from utils.ml_logging import get_logger

# Load environment variables from .env file
load_dotenv()
# Set up logger
logger = get_logger()

# Create an instance of the client. You can find it in src/aoai/azure_openai.py.
# It is essentially a wrapper using dependency injection to automate the initialization
# and most used API calls.
azure_openai_client = AzureOpenAIManager()

In [4]:
azure_openai_client.generate_chat_response(conversation_history=[], query="Deep Dive in the transformae architecture and focus on intrsucto -GPT", max_tokens=1000)

2024-03-08 15:46:10,269 - micro - MainProcess - INFO     Sending request to OpenAI with query: Deep Dive in the transformae architecture and focus on intrsucto -GPT (azure_openai.py:generate_chat_response:190)


2024-03-08 15:46:10,510 - micro - MainProcess - ERROR    Another non-200-range status code was received (azure_openai.py:generate_chat_response:218)
2024-03-08 15:46:10,512 - micro - MainProcess - ERROR    404 (azure_openai.py:generate_chat_response:219)
2024-03-08 15:46:10,513 - micro - MainProcess - ERROR    <Response [404 Not Found]> (azure_openai.py:generate_chat_response:220)


In [2]:
#Note: The openai-python library support for Azure OpenAI is in preview.
      #Note: This code sample requires OpenAI Python library version 1.0.0 or higher.
import os
from openai import AzureOpenAI


client = AzureOpenAI(
  azure_endpoint = "https://gbb-ea-openai-southcentralus-01.openai.azure.com/", 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)


message_text = [{"role":"system","content":"You are an AI assistant that helps people find information."},{"role":"user","content":"Please compare Kentucky vs Madrid"},{"role":"assistant","content":"Comparing Kentucky (a state in the United States) and Madrid (the"}]

completion = client.chat.completions.create(
  model="gpt-4-0125-ptu", # model = "deployment_name"
  messages = message_text,
  temperature=0.7,
  max_tokens=3000,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

In [3]:
len(completion.choices[0].message.content)

3017

In [9]:
import pprint
pprint.pprint(completion.choices[0].message.content)

('Comparing Kentucky, a state in the United States, with Madrid, the capital '
 'city of Spain, involves looking at several aspects such as geographical '
 'location, climate, economy, culture, and tourism. Here is a comparative '
 'overview:\n'
 '\n'
 '**Geographical Location and Size:**\n'
 '- **Kentucky:** Located in the southeastern region of the United States, '
 'Kentucky is a state that covers an area of about 40,409 square miles '
 '(104,659 square kilometers).\n'
 '- **Madrid:** Madrid is both a city and a region (autonomous community) '
 'within Spain. The city is in the center of the country and serves as the '
 'capital. The Madrid region covers an area of approximately 3,083 square '
 'miles (7,995 square kilometers).\n'
 '\n'
 '**Climate:**\n'
 '- **Kentucky:** Kentucky experiences a humid subtropical climate, with hot, '
 "humid summers and cold, moderately snowy winters. The state's weather can "
 'vary significantly from one region to another.\n'
 '- **Madrid:** Madrid

In [4]:
from src.aoai.tokenizer import AzureOpenAITokenizer
az_tokenizer = AzureOpenAITokenizer()

In [5]:
az_tokenizer.estimate_tokens_completion(response=completion.choices[0].message.content, model="gpt-4")

609

In [16]:
import time

def run_and_get_stats():
    stats = []
    for i in range(10):
        start_time = time.time()

        completion = client.chat.completions.create(
            model="gpt-4-0125-ptu",
            messages=message_text,
            temperature=0.7,
            max_tokens=3000,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None
        )

        tokens = az_tokenizer.estimate_tokens_completion(response=completion.choices[0].message.content, model="gpt-4")
        message_length = len(completion.choices[0].message.content)
        run_time = time.time() - start_time

        stats.append({
            'run': i + 1,
            'tokens': tokens,
            'message_length': message_length,
            'run_time': run_time,
        })

    return stats

# Run the function and print the stats
stats = run_and_get_stats()

In [18]:
from tabulate import tabulate
from termcolor import colored

total_tokens = 0
total_run_time = 0
total_message_length = 0
table = []
headers = ["Run", "Tokens", "Message Length", "Run Time (seconds)"]

for stat in stats:
    total_tokens += stat['tokens']
    total_run_time += stat['run_time']
    total_message_length += stat['message_length']
    table.append([stat['run'], stat['tokens'], stat['message_length'], stat['run_time']])

average_tokens = total_tokens / len(stats)
average_run_time = total_run_time / len(stats)
average_message_length = total_message_length / len(stats)

table.append(["Average", average_tokens, average_message_length, average_run_time])

print(colored(tabulate(table, headers, tablefmt="pretty"), 'cyan'))

+---------+--------+----------------+--------------------+
|   Run   | Tokens | Message Length | Run Time (seconds) |
+---------+--------+----------------+--------------------+
|    1    |  617   |      3050      | 15.408635139465332 |
|    2    |  577   |      2915      | 14.157102584838867 |
|    3    |  572   |      2870      |  14.0344557762146  |
|    4    |  597   |      3073      | 14.581034183502197 |
|    5    |  710   |      3632      | 17.40005922317505  |
|    6    |  591   |      2923      | 14.38428258895874  |
|    7    |  631   |      3115      | 15.41484022140503  |
|    8    |  738   |      3763      | 18.071146488189697 |
|    9    |  620   |      3148      | 15.073824405670166 |
|   10    |  641   |      3309      | 15.644548177719116 |
| Average | 629.4  |     3179.8     | 15.416992878913879 |
+---------+--------+----------------+--------------------+
